In [1]:
!pip install elasticsearch
!pip install pymongo

In [1]:
from pymongo import MongoClient

In [2]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

import pandas as pd
import json


import numpy as np

LOCAL = True

es_client = Elasticsearch(hosts=["localhost" if LOCAL else "elasticsearch"])

#df_billboard = pd.read_json("./newscrawler/newscrawler/spiders/scrap200.json")

In [3]:
client = MongoClient()
db = client["client_name"]
billboard = db["billboard"]

In [4]:
df_billboard = pd.DataFrame(list(billboard.find()))
df_billboard.head()

,_id,album,artist,rank
0,5e4f97c8cede4c262f788206,Please Excuse Me For Being Antisocial,Roddy Ricch,1
1,5e4f97c8cede4c262f788207,Hollywood's Bleeding,Post Malone,2
2,5e4f97c8cede4c262f788208,Music To Be Murdered By,Eminem,3
3,5e4f97c8cede4c262f788209,Father Of All...,Green Day,4
4,5e4f97c8cede4c262f78820a,"When We All Fall Asleep, Where Do We Go?",Billie Eilish,5


In [5]:
df_billboard_clean = df_billboard.drop(labels='_id',axis='columns')
documents = df_billboard_clean.fillna("").to_dict(orient="records")

In [6]:
documents[0:1]

[{'album': 'Please Excuse Me For Being Antisocial',
  'artist': 'Roddy Ricch',
  'rank': '1'}]

In [7]:
len(documents)

200

In [8]:
def generate_data(documents):
    for docu in documents:
        yield {
            "_index": "albums",
            "_type": "album",
            #"_source": {k:v if v else None for k,v in docu.items()},
        }

bulk(es_client, generate_data(documents))

(200, [])

In [9]:
#!curl http://localhost:9200/_cat/indices?v

In [10]:
#!curl http://localhost:9200/albums/_search

In [11]:
QUERY = {
  "query": {
    "multi_match" : {
      "query":    "halsey",
      "fields": [ "artist" ] 
    }
  }
}

In [12]:
result = es_client.search(index="albums", body=QUERY)
[elt['album'] for elt in result["hits"]]

TypeError: string indices must be integers

In [13]:
len(result)

4

In [14]:
result["hits"]["hits"]

[{'_index': 'albums',
  '_type': 'album',
  '_id': 'T4IEZ3ABDoZpy6JhEJRi',
  '_score': 6.6659617,
  '_source': {'album': 'Manic', 'artist': 'Halsey', 'rank': '8'}},
 {'_index': 'albums',
  '_type': 'album',
  '_id': 'L4ICZ3ABDoZpy6JhfpEA',
  '_score': 6.6659617,
  '_source': {'album': 'Manic', 'artist': 'Halsey', 'rank': '8'}},
 {'_index': 'albums',
  '_type': 'album',
  '_id': 'N4IdZ3ABDoZpy6JhIphM',
  '_score': 6.6659617,
  '_source': {'album': 'Manic', 'artist': 'Halsey', 'rank': '8'}},
 {'_index': 'albums',
  '_type': 'album',
  '_id': 'b4IXZ3ABDoZpy6JhjZcE',
  '_score': 6.6659617,
  '_source': {'album': 'Manic', 'artist': 'Halsey', 'rank': '8'}},
 {'_index': 'albums',
  '_type': 'album',
  '_id': '94ICZ3ABDoZpy6JhmpHr',
  '_score': 6.6659617,
  '_source': {'album': 'Manic', 'artist': 'Halsey', 'rank': '8'}},
 {'_index': 'albums',
  '_type': 'album',
  '_id': '34IFZ3ABDoZpy6JhCZWw',
  '_score': 6.6659617,
  '_source': {'album': 'Manic', 'artist': 'Halsey', 'rank': '8'}},
 {'_index'

In [ ]:
for el['_id'] in result["hits"]:
    

In [ ]:
{'_index': 'albums',
  '_type': 'album',
  '_id': 'L4ICZ3ABDoZpy6JhfpEA',
  '_score': 6.6593313,
  '_source': {'album': 'Manic', 'artist': 'Halsey', 'rank': '8'}},

In [54]:
result["hits"]["hits"]

[{'_index': 'albums',
  '_type': 'album',
  '_id': 'L4ICZ3ABDoZpy6JhfpEA',
  '_score': 6.6593313,
  '_source': {'album': 'Manic', 'artist': 'Halsey', 'rank': '8'}},
 {'_index': 'albums',
  '_type': 'album',
  '_id': '94ICZ3ABDoZpy6JhmpHr',
  '_score': 6.6593313,
  '_source': {'album': 'Manic', 'artist': 'Halsey', 'rank': '8'}},
 {'_index': 'albums',
  '_type': 'album',
  '_id': 'v4IDZ3ABDoZpy6Jhr5LJ',
  '_score': 6.6593313,
  '_source': {'album': 'Manic', 'artist': 'Halsey', 'rank': '8'}},
 {'_index': 'albums',
  '_type': 'album',
  '_id': 'h4IDZ3ABDoZpy6Jh2ZPX',
  '_score': 6.6593313,
  '_source': {'album': 'Manic', 'artist': 'Halsey', 'rank': '8'}},
 {'_index': 'albums',
  '_type': 'album',
  '_id': 'T4IEZ3ABDoZpy6JhEJRi',
  '_score': 6.6593313,
  '_source': {'album': 'Manic', 'artist': 'Halsey', 'rank': '8'}},
 {'_index': 'albums',
  '_type': 'album',
  '_id': 'F4IEZ3ABDoZpy6JhUpVz',
  '_score': 6.6593313,
  '_source': {'album': 'Manic', 'artist': 'Halsey', 'rank': '8'}},
 {'_index'

In [52]:
!python3 music_search.py

 * Serving Flask app "music_search" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:2745/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 107-460-003
127.0.0.1 - - [19/Feb/2020 18:10:32] "GET / HTTP/1.1" 302 -
music_search.py:38: FlaskWTFDeprecationWarning: "flask_wtf.Form" has been renamed to "FlaskForm" and will be removed in 1.0.
  form = SearchBar()
127.0.0.1 - - [19/Feb/2020 18:10:32] "GET /MusicSearch HTTP/1.1" 200 -
music_search.py:38: FlaskWTFDeprecationWarning: "flask_wtf.Form" has been renamed to "FlaskForm" and will be removed in 1.0.
  form = SearchBar()
127.0.0.1 - - [19/Feb/2020 18:10:34] "POST /MusicSearch HTTP/1.1" 302 -
127.0.0.1 - - [19/Feb/2020 18:10:34] "GET /information/Taylor%20Swift HTTP/1.1" 500 -
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/flask/app.py", line 2309, in __call__
    return self.wsgi_app(environ, 